### Import libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import glob
import os
import feather
sns.set(style="whitegrid")

--------------------------------------

### Load calculated features

In [ ]:
fullFeatMat = feather.read_dataframe('data/EmpFeatMat.feather')
filt        = feather.read_dataframe('data/fullFeatMatFilt.feather')

In [ ]:
tsfel = filt[filt['method'] == 'tsfel']
tsfresh = filt[filt['method'] == 'tsfresh']

# add column comb_id
tsfel['comb_id']   = tsfel.apply(lambda row: row['method'] + '_' + row['names'], axis=1)
tsfresh['comb_id'] = tsfresh.apply(lambda row: row['method'] + '_' + row['names'], axis=1)


In [ ]:
def withinCorrelation(dataset):
    
    # each column represents the value of a feature for all the given time series'
    pivotTable = pd.pivot_table(dataset, index=['id'], columns=['comb_id'])
    
    # get the spearman correlation - feature by feature
    df = pivotTable.corr(method='spearman')

    tmpDF = df.stack()
    tmpDF.index.names = [None, 'V1', 'V2']
    tmpDF.columns = ['correlation']
    return tmpDF.reset_index().iloc[: , 1:], df.iloc[:, ::-1]
    

In [ ]:
tsfel_corrFrame, tsfel_corrMatrix      = withinCorrelation(tsfel)
tsfresh_corrFrame, tsfresh_corrMatrix  = withinCorrelation(tsfresh)

In [ ]:
l = len(tsfel_corrFrame['V1'].unique()) * [None]
l[0] = 'F1'
l[99] = 'F100'
l[199] = 'F200'
l[299] = 'F300'
l[377] = 'F378'
l.reverse()
p = l.copy()
l.reverse()

In [ ]:
ax = plt.axes()
featBfeat = sns.heatmap(tsfel_corrMatrix, ax = ax, cmap='viridis', xticklabels = l, yticklabels = p)

yticks = np.linspace(10,100,10)
ylabels = np.linspace(100,378,5)


#ax.set_title('Feature by feature correlation - TSFEL')
ax.set(xlabel=None, ylabel=None)
plt.show()

fig = featBfeat.get_figure()
fig.savefig('outputs/featByFeat_tsfel.pdf')

In [ ]:
len(tsfresh_corrFrame['V1'].unique())

In [ ]:
l = len(tsfresh_corrFrame['V1'].unique()) * [None]
l[0] = 'F1'
l[149] = 'F150'
l[299] = 'F300'
l[449] = 'F450'
l[599] = 'F600'
l[743] = 'F744'
l.reverse()
p = l.copy()
l.reverse()

In [ ]:
ax = plt.axes()
featBfeat = sns.heatmap(tsfresh_corrMatrix, ax = ax, cmap='viridis', yticklabels=p, xticklabels=l)

#ax.set_title('Feature by feature correlation - TSFRESH')
ax.set(xlabel=None, ylabel=None)
plt.show()

fig = featBfeat.get_figure()
fig.savefig('outputs/featByFeat_tsfresh.pdf')

--------------------------------------